In [ ]:
docker rm -f sql
docker run -e "ACCEPT_EULA=Y" -e "SA_PASSWORD=MyPass@word" -e "MSSQL_PID=Express" -p 1433:1433 -d --name=sql mcr.microsoft.com/mssql/server:latest

In [1]:
docker ps --filter="name=sql"

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


In [ ]:
if (!$OriginalProjectPath) {$OriginalProjectPath = (Get-Item .).FullName}
$Env:terraformworkingdir = $OriginalProjectPath + '\WorkingDirectory\LocalDockerExample'
Write-Output "OriginalProjectPath: $OriginalProjectPath"
Write-Output "WorkingDirectory: $Env:terraformworkingdir"
$WorkingDirectory = $Env:terraformworkingdir

In [ ]:
cd $env:TEMP\tf
$TerraformExePath = (Get-ChildItem)[0].FullName


In [ ]:
"TerraformExePath: $TerraformExePath"

In [ ]:
.\terraform version

In [ ]:
$command = 'docker inspect --format="{{.ID}}" sql'
Write-Output "Executing the command:"
Write-Output $command
$DockerId = Invoke-Expression $command
$Env:ImportDockerId = $DockerId
Write-Output 'Docker Id in question:' $Env:ImportDockerId

In [ ]:
$content = Get-Content -Path "$Env:terraformworkingdir\LocalDockerExampleOriginal.tf.bak"
$content | Set-Content "$Env:terraformworkingdir\LocalDockerExample.tf"

In [ ]:
.\terraform --% -chdir=%terraformworkingdir% init

In [ ]:
.\terraform --% -chdir=%terraformworkingdir% import docker_container.arbitrarylabel %ImportDockerId%

In [ ]:
.\terraform --% -chdir=%terraformworkingdir% show

In [ ]:
.\terraform --% -chdir=%terraformworkingdir% plan
Write-Output "****** Errors are Expected ******"

In [ ]:
$command = ".\terraform -chdir=$WorkingDirectory show -no-color > $WorkingDirectory\LocalDockerExample.tf"
Write-Output "Executing the command:"
Write-Output $command
Invoke-Expression $command

In [ ]:
(Get-Content -Raw "$WorkingDirectory\LocalDockerExample.tf")

In [ ]:
$NewContent = (
    @'
terraform {
    required_providers {
        docker = {
            source  = "kreuzwerker/docker"
            version = "~> 2.17.0"
        }
    }
}
provider "docker" {
  #host = "unix:///var/run/docker.sock"
  host    = "npipe:////.//pipe//docker_engine"
}

'@).ReplaceLineEndings() + (Get-Content -Raw -Path "$WorkingDirectory\LocalDockerExample.tf").ReplaceLineEndings()
$NewContent = $NewContent.Replace('gateway           =',"# gateway           =")
$NewContent = $NewContent.Replace('id                =',"# id                =")
$NewContent = $NewContent.Replace('ip_address        =',"# ip_address        =")
$NewContent = $NewContent.Replace('ip_prefix_length  =',"# ip_prefix_length  =")
$NewContent = $NewContent.Replace('network_data      =',"/*network_data      =")
$NewContent = $NewContent.Replace('network_mode      =',"*/`n    network_mode      =")
$NewContent | Set-Content "$WorkingDirectory\LocalDockerExample.tf"

In [ ]:
(Get-Content -Raw "$WorkingDirectory\LocalDockerExample.tf")

In [ ]:
.\terraform --% -chdir=%terraformworkingdir% init -upgrade -input=false
Write-Output "****** Errors are NOT Expected ******"

In [ ]:
.\terraform --% -chdir=%terraformworkingdir% plan -input=false
Write-Output "****** Errors are NOT Expected ******"

In [ ]:
.\terraform --% -chdir=%terraformworkingdir% plan -out=tfplan -input=false
Write-Output "****** Errors are NOT Expected ******"

In [ ]:
.\terraform --% -chdir=%terraformworkingdir% apply -input=false tfplan

In [ ]:
$NewContent = (Get-Content -Raw -Path "$WorkingDirectory\LocalDockerExample.tf").ReplaceLineEndings()
$NewContent = $NewContent.Replace('    ports {', @"
    upload {
        content = "${file("sample.txt")}"
        file = "/var/sample.txt"
    }
    ports {
"@)
$NewContent | Set-Content "$WorkingDirectory\LocalDockerExample.tf"

In [ ]:
# Full Reset
if (!$WorkingDirectory) {exit} # Guard Clause
#Remove-Item -LiteralPath "$WorkingDirectory\.terraform" #-Force -Recurse -whatif

In [ ]:
.\terraform --% -chdir=%terraformworkingdir% destroy -auto-approve

[How to Deploy to a remote Docker](https://www.phillipsj.net/posts/yes-terraform-can-do-that-docker/)